# Functional test with FTDI
- https://www.silabs.com/documents/public/data-sheets/Si5351-B.pdf
- https://www.silabs.com/documents/public/application-notes/AN619.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', 'ORM', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', 'USB', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from clock_generators.si535x.si5351 import Si5351 
from utilities.adapters import peripherals

Populating the interactive namespace from numpy and matplotlib
No USB device matches URL ftdi://ftdi:ft232h/1
No USB device matches URL ftdi://ftdi:ft232h/1


## Debug mode?

In [5]:
cls = Si5351

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [6]:
with_hardware_device = False

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c() 
    
else:
    _i2c =  None  # using None for testing without actual hardware device.

si = cls(_i2c)  

****** Virtual device. Data may not be real ! ******


In [7]:
matchs = si.find_integer_dividers(freq_desired = 45e6, even_only = True, torance_hz = 1) 
matchs

[((36, 10, 2), (25000000, 900000000, 90000000, 45000000)),
 ((36, 20, 1), (25000000, 900000000, 45000000, 45000000))]

In [8]:
desired_clock_freqs = (25e6, 25e5, 30e6)

common_pll_dividers, freqs_pll_dividers, freqs_matches = si.find_integer_pll_dividers_for_clocks(desired_clock_freqs)
common_pll_dividers, freqs_pll_dividers

({24, 36},
 [{24, 26, 28, 30, 32, 34, 36}, {24, 26, 28, 30, 32, 34, 36}, {24, 36}])

In [9]:
clk = si.clocks[0]

In [10]:
clk.set_frequency(25e6)

True

In [27]:
clk.freq

25000000

In [25]:
si.plls[0].set_frequency(900e6)

True

In [26]:
si.plls[0].divider

36.0

In [14]:
# _i2c.scan()

In [15]:
si.enable(True)

In [16]:
si.enable(False)

In [17]:
si.clocks[0].enable()

In [18]:
si.read_all_registers()

[(0, 'Device_Status', 0),
 (1, 'Interrupt_Status_Sticky', 0),
 (2, 'Interrupt_Status_Mask', 0),
 (3, 'Output_Enable_Control', 0),
 (9, 'OEB_Pin_Enable_Control_Mask', 0),
 (15, 'PLL_Input_Source', 0),
 (16, 'CLK0_Control', 0),
 (17, 'CLK1_Control', 0),
 (18, 'CLK2_Control', 0),
 (19, 'CLK3_Control', 0),
 (20, 'CLK4_Control', 0),
 (21, 'CLK5_Control', 0),
 (22, 'CLK6_Control', 0),
 (23, 'CLK7_Control', 0),
 (24, 'CLK3_0_Disable_State', 0),
 (25, 'CLK7_4_Disable_State', 0),
 (26, 'Multisynth_NA_Parameters_26', 0),
 (27, 'Multisynth_NA_Parameters_27', 0),
 (28, 'Multisynth_NA_Parameters_28', 0),
 (29, 'Multisynth_NA_Parameters_29', 0),
 (30, 'Multisynth_NA_Parameters_30', 0),
 (31, 'Multisynth_NA_Parameters_31', 0),
 (32, 'Multisynth_NA_Parameters_32', 0),
 (33, 'Multisynth_NA_Parameters_33', 0),
 (34, 'Multisynth_NB_Parameters_34', 0),
 (35, 'Multisynth_NB_Parameters_35', 0),
 (36, 'Multisynth_NB_Parameters_36', 0),
 (37, 'Multisynth_NB_Parameters_37', 0),
 (38, 'Multisynth_NB_Parameters_

In [19]:
si.map.addressed_values

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (9, 0),
 (15, 0),
 (16, 0),
 (17, 0),
 (18, 0),
 (19, 0),
 (20, 0),
 (21, 0),
 (22, 0),
 (23, 0),
 (24, 0),
 (25, 0),
 (26, 0),
 (27, 0),
 (28, 0),
 (29, 0),
 (30, 0),
 (31, 0),
 (32, 0),
 (33, 0),
 (34, 0),
 (35, 0),
 (36, 0),
 (37, 0),
 (38, 0),
 (39, 0),
 (40, 0),
 (41, 0),
 (42, 0),
 (43, 0),
 (44, 0),
 (45, 0),
 (46, 0),
 (47, 0),
 (48, 0),
 (49, 0),
 (50, 0),
 (51, 0),
 (52, 0),
 (53, 0),
 (54, 0),
 (55, 0),
 (56, 0),
 (57, 0),
 (58, 0),
 (59, 0),
 (60, 0),
 (61, 0),
 (62, 0),
 (63, 0),
 (64, 0),
 (65, 0),
 (66, 0),
 (67, 0),
 (68, 0),
 (69, 0),
 (70, 0),
 (71, 0),
 (72, 0),
 (73, 0),
 (74, 0),
 (75, 0),
 (76, 0),
 (77, 0),
 (78, 0),
 (79, 0),
 (80, 0),
 (81, 0),
 (82, 0),
 (83, 0),
 (84, 0),
 (85, 0),
 (86, 0),
 (87, 0),
 (88, 0),
 (89, 0),
 (90, 0),
 (91, 0),
 (92, 0),
 (149, 0),
 (150, 0),
 (151, 0),
 (152, 0),
 (153, 0),
 (154, 0),
 (155, 0),
 (156, 0),
 (157, 0),
 (158, 0),
 (159, 0),
 (160, 0),
 (161, 0),
 (162, 0),
 (163, 0),
 (164, 0),

In [20]:
si.interrupts.masks.print()


<< Interrupt_Status_Mask >>:  ('0x0', '0b0')
[ SYS_INIT_MASK ]   :  0
[ LOL_B_MASK ]      :  0
[ LOL_A_MASK ]      :  0
[ LOS__CLKIN_MASK ] :  0
[ LOS__XTAL_MASK ]  :  0
[ Reserved_0 ]      :  0


15

In [21]:
si.interrupts.stickys.print()


<< Interrupt_Status_Sticky >>:  ('0x0', '0b0')
[ SYS_INIT_STKY ]  :  0
[ LOL_B_STKY ]     :  0
[ LOL_A_STKY ]     :  0
[ LOS_CLKIN_STKY ] :  0
[ LOS_XTAL_STKY ]  :  0
[ Reserved_0 ]     :  0


14

## Member functions test

In [22]:
for f in dir(cls):
    if not f.startswith('_'):
        print('si.{}()'.format(f))

si.CLKIN_DIVIDERS()
si.CLOCK_SOURCEs()
si.CRYSTAL_INTERNAL_LOAD_CAPACITANCEs()
si.DEBUG_MODE()
si.DEBUG_MODE_PRINT_REGISTER()
si.DEBUG_MODE_SHOW_BUS_DATA()
si.DENOMINATOR_BITS()
si.DISABLE_STATEs()
si.FREQ_MCLK()
si.I2C_ADDRESS()
si.INTEGER_ONLY_MULTISYNTHES()
si.N_OUTPUT_CLOCKS()
si.N_PLLS()
si.N_USED_OUTPUT_CLOCKS()
si.OUTPUT_STRENGTHs()
si.PLLs()
si.POW_2_DENOMINATOR_BITS()
si.READ_ONLY_REGISTERS()
si.R_DIVIDERs()
si.close()
si.current_frequency()
si.current_phase()
si.do()
si.do_on_devices()
si.enable()
si.enable_output()
si.enable_output_channel()
si.find_integer_dividers()
si.find_integer_pll_dividers_for_clocks()
si.init()
si.load_registers()
si.pause()
si.print()
si.read_all_registers()
si.reset()
si.restore_clocks_freqs()
si.resume()
si.set_frequency()
si.set_phase()
si.start()
si.status()
si.stop()
si.update()
si.write_all_registers()


In [23]:
si.apply_signal()
si.current_frequency()
si.current_phase()
si.do()
si.do_on_devices()
si.enable()
si.enable_output()
si.enable_output_channel()
si.enable_output_integer_divider()
si.enable_phase_offset()
si.enable_pll_integer_divider()
si.enable_spread_spectrum()
si.enabled()
si.freq_resolution()
si.frequency()
si.get_freq_vco_clkin()
si.get_freq_vco_xtal()
si.get_multisynth_nb_parameter()
si.get_output_multisynth_divider()
si.get_pll_multisynth_divider()
si.init()
si.is_abc_even_integer()
si.is_even_integer()

si.phase_resolution()
si.power_down_all_outputs()
si.power_down_clock_output()
si.power_down_output()
si.print()
si.reset()
si.reset_pll()
si.reset_plls()

si.select_freq_source()
si.select_phase_source()
si.set_center_spread()
si.set_clock_disable_state()
si.set_clock_invert()
si.set_cmos_clock_source()
si.set_divide_by_4()
si.set_down_spread()
si.set_frequency()
si.set_interrupt_mask()
si.set_interrupts_mask()
si.set_output_driver_source()
si.set_output_multisynth_source()
si.set_phase()
si.set_pll_input_source()
si.set_r_divider()
si.set_ss_parameters()
si.set_vcoa()
si.set_vcob()
si.set_vcxo_paramenters()
si.set_xtal_source()
si.shape()
si.start()
si.pause()
si.resume()
si.stop()
si.update()

si.close()

AttributeError: 'Si5351' object has no attribute 'apply_signal'

In [ ]:
si.phase_resolution()